In [1]:
#Загружаем нужные библиотеки

import pandas as pd

from time import sleep

from tqdm import tqdm_notebook

from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.common.exceptions import NoSuchElementException

import os
import re

In [4]:
# вытаскиваем названия файлов в список file_list

path_ula = 'C:/00_Projects/Rus_mushrooms/Parsing_results/Ula/'
file_list = []


for root, dirs, files in os.walk(path_ula):
    
    for filename in files:
        file_list += [filename]      

In [5]:
%%time

# преобразовываю и объединяю в один датафрейм

mushrooms_ula = pd.DataFrame()

for i in file_list:
    temp = pd.read_excel(path_avito + i)
    
    mushrooms_ula = mushrooms_ula.append(temp, ignore_index = True)

Wall time: 281 ms


In [7]:
mushrooms_ula.shape

(1733, 7)

In [8]:
mushrooms_ula.sample(3)

,topic,city,price,ads_text,seller,user_id,ads_url
393,Чайный гриб,Псков,300 ₽,https://youla.ru/user/58b439925bd2e92375f04f7f,Маргарита Ш. (3 объявления),https://youla.ru/user/58b439925bd2e92375f04f7f,https://youla.ru/pskov/uslugi/produkty-pitaniy...
1128,нет,нет,нет,нет,Антисептик Москва (1729 объявлений),https://youla.ru/user/5f1155d51f2ccf35a328a1f8,https://youla.ru/moskva/krasota-i-zdorove/sred...
881,Антисептик для рук,Москва,50 ₽,https://youla.ru/user/5f1155d51f2ccf35a328a1f8,Антисептик Москва (1729 объявлений),https://youla.ru/user/5f1155d51f2ccf35a328a1f8,https://youla.ru/moskva/krasota-i-zdorove/sred...


In [9]:
mushrooms_ula = mushrooms_ula.drop_duplicates()

In [10]:
mushrooms_ula.shape

(1604, 7)

In [12]:
# меняю ё на е

mushrooms_ula.topic = mushrooms_ula.topic.str.replace('ё', 'е')
mushrooms_ula.ads_text = mushrooms_ula.ads_text.str.replace('ё', 'е')

C:\Users\Boris\Anaconda3\lib\site-packages\pandas\core\generic.py:5165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [13]:
# список грибов (включая чайный и рейши с кордицепсом)

all_mushrooms = [
    'amanita',
    'chaga',
    'ganoderma',
    'grifola',
    'maitake',
    'reish',
    'shiitake',
    'агарик бразильский',
    'агарикус',
    'аманита',
    'антродия',
    'бeлые',
    'баран',
    'белого',
    'белые',
    'белый',
    'белыми',
    'белых',
    'белянк',
    'боровик',
    'бычки',
    'валуи',
    'валуй',
    'веселк',
    'веселуш',
    'вешенк',
    'волжанк',
    'волнух',
    'волнуш',
    'вороночник',
    'ганодерм',
    'гериций',
    'герициум',
    'гигрофор',
    'гладушк',
    'гладыш',
    'говорушк',
    'головач',
    'горькушк',
    'гребенчатый',
    'грузд',
    'дождевик',
    'дорогие',
    'дубовик',
    'дуники',
    'дуньк',
    'ежевик',
    'ежовик',
    'еловик',
    'забалуйк',
    'заболуйк',
    'зеленух',
    'зеленушк',
    'зонтик',
    'камбуч',
    'колпак',
    'комбуч',
    'кордицепс',
    'коровник',
    'красноголовик',
    'кубар',
    'кубышк',
    'кулаки',
    'кулачки',
    'кулачок',
    'кулочки',
    'линчжи',
    'липовк',
    'лисичк',
    'лубянк',
    'майка',
    'майтак',
    'майтаке',
    'масленок',
    'маслят',
    'мацутак',
    'медвежье',
    'мейтаке',
    'микродозинг',
    'млечник',
    'молочник',
    'морозик',
    'мослят',
    'моховик',
    'мухомор',
    'навозник',
    'обабк',
    'опенок',
    'опят',
    'пантерн',
    'паутиник',
    'паутинник',
    'песочник',
    'песчаник',
    'поганк',
    'податавник',
    'подберезовик',
    'подгрузок',
    'поддубник',
    'поддубовик',
    'подорешник',
    'подосиновик',
    'подотавник',
    'подтопол',
    'полубел.',
    'польский',
    'попутк',
    'попутниц',
    'рейш',
    'рыжик',
    'рядовк',
    'свинар',
    'свину',
    'серух',
    'серушк',
    'серянк',
    'синеножк',
    'синявк',
    'синяя ножка',
    'сморч',
    'солонушк',
    'соплюшки',
    'строфария',
    'строч',
    'сыраешк',
    'сыроежк',
    'трамет',
    'трутовик берез.',
    'трутовик лакированный',
    'трутовик лиственничный',
    'трутовик ложный дубовый',
    'трутовик настоящий',
    'трутовик неизв.',
    'трутовик окаймленный',
    'трутовик осиновый',
    'трутовик плоский',
    'трутовик серно-желтый',
    'трутовик смолистый',
    'трутовик сосновый',
    'трутовик яблоневый',
    'трюфел',
    'турки',
    'фламмулин',
    'чаг',
    'чайный',
    'чернушк',
    'чесночник',
    'чешутчатк',
    'чинарик',
    'шампин',
    'шиитак',
    'шитаки',
    'эноки',
    'ярсагумба'
]

In [14]:
len(all_mushrooms)

154

In [15]:
# словарь грибов: словоформа или синоним — название гриба (понадобится позже)

mushrooms_dict = {
    'amanita': 'мухомор',
    'chaga': 'чага',
    'ganoderma': 'трутовик лакированный (рейши)',
    'grifola': 'грифола',
    'maitake': 'грифола',
    'reish': 'трутовик лакированный (рейши)',
    'shiitake': 'шиитаке',
    'агарик бразильский': 'агарик бразильский',
    'агарикус': 'трутовик лиственничный',
    'аманита': 'мухомор',
    'антродия': 'антродия',
    'бeлые': 'белый',
    'баран': 'грифола',
    'белого': 'белый',
    'белые': 'белый',
    'белый': 'белый',
    'белыми': 'белый',
    'белых': 'белый',
    'белянк': 'волнушка белая',
    'боровик': 'белый',
    'бычки': 'валуй',
    'валуи': 'валуй',
    'валуй': 'валуй',
    'веселк': 'веселка',
    'веселуш': 'веселка',
    'вешенк': 'вешенка',
    'волжанк': 'волнушка розовая',
    'волнух': 'волнушка',
    'волнуш': 'волнушка',
    'вороночник': 'вороночник',
    'ганодерм': 'трутовик лакированный (рейши)',
    'гериций': 'ежовик',
    'герициум': 'ежовик',
    'гигрофор': 'гигрофор',
    'гладушк': 'млечник',
    'гладыш': 'млечник',
    'говорушк': 'говорушка (рядовка)',
    'головач': 'головач (шампиньон)',
    'горькушк': 'горькушка (рядовка)',
    'гребенчатый': 'ежовик',
    'грузд': 'груздь',
    'дождевик': 'дождевик',
    'дорогие': 'белый',
    'дубовик': 'дубовик',
    'дуники': 'свинушка тонкая',
    'дуньк': 'свинушка тонкая',
    'ежевик': 'ежовик',
    'ежовик': 'ежовик',
    'еловик': 'еловик',
    'забалуйк': 'рядовка тополевая',
    'заболуйк': 'рядовка тополевая',
    'зеленух': 'зеленушка (рядовка)',
    'зеленушк': 'зеленушка (рядовка)',
    'зонтик': 'зонтик',
    'камбуч': 'чайный гриб',
    'колпак': 'колпак кольчатый',
    'комбуч': 'чайный гриб',    
    'кордицепс': 'кордицепс',
    'коровник': 'свинушка тонкая',
    'красноголовик': 'подосиновик',
    'кубар': 'валуй',
    'кубышк': 'валуй',
    'кулаки': 'валуй',
    'кулачки': 'валуй', 
    'кулачок': 'валуй',
    'кулочки': 'валуй',
    'линчжи': 'трутовик лакированный (рейши)',
    'липовк': 'липовка',
    'лисичк': 'лисичка',
    'лубянк': 'горькушка (рядовка)',
    'майка': 'рядовка майская',
    'майтак': 'грифола',
    'майтаке': 'грифола',
    'масленок': 'масленок',
    'маслят': 'масленок',
    'мацутак': 'мацутакэ',
    'медвежье': 'медвежье ушко',
    'мейтаке': 'грифола',
    'микродозинг': 'мухомор',
    'млечник': 'млечник',
    'молочник': 'млечник',
    'морозик': 'рядовка тополевая',
    'мослят': 'масленок',
    'моховик': 'моховик',
    'мухомор': 'мухомор',
    'навозник': 'навозник',
    'обабк': 'обабок',
    'опенок': 'опенок',
    'опят': 'опенок',
    'пантерн': 'мухомор', 
    'паутиник': 'паутинник',
    'паутинник': 'паутинник',
    'песочник': 'рядовка тополевая',
    'песчаник': 'рядовка тополевая',
    'поганк': 'бледная поганка',
    'податавник': 'рядовка лиловоногая',
    'подберезовик': 'подберезовик',
    'подгрузок': 'подгруздок (сыроежка)',
    'поддубник': 'дубовик',
    'поддубовик': 'дубовик',
    'подорешник': 'подорешник',
    'подосиновик': 'подосиновик',
    'подотавник': 'рядовка лиловоногая',
    'подтопол': 'рядовка тополевая',
    'полубел.': 'полубелый',
    'польский': 'польский гриб',
    'попутк': 'серушка',
    'попутниц': 'серушка',
    'рейш': 'трутовик лакированный (рейши)',
    'рыжик': 'рыжик',
    'рядовк': 'рядовка',
    'свинар': 'свинушка тонкая',
    'свину': 'свинушка тонкая',
    'серух': 'серушка',
    'серушк': 'серушка',
    'серянк': 'серушка',
    'синеножк': 'рядовка лиловоногая',
    'синявк': 'сыроежка',
    'синяя ножка': 'рядовка лиловоногая',
    'сморч': 'сморчок',
    'солонушк': 'волнушка',
    'соплюшки': 'валуй',
    'строфария': 'строфария',
    'строч': 'строчок',
    'сыраешк': 'сыроежка',
    'сыроежк': 'сыроежка',
    'трамет': 'трутовик разноцветный',
    'трутовик берез.': 'трутовик березовый',
    'трутовик лакированный': 'трутовик лакированный (рейши)',
    'трутовик лиственничный': 'трутовик лиственничный',
    'трутовик ложный дубовый': 'трутовик ложный дубовый',
    'трутовик настоящий': 'трутовик окаймленный',
    'трутовик неизв.': 'просто трутовик',
    'трутовик окаймленный': 'трутовик окаймленный',
    'трутовик осиновый': 'трутовик осиновый',
    'трутовик плоский': 'трутовик плоский',
    'трутовик серно-желтый': 'трутовик серно-желтый',
    'трутовик смолистый': 'трутовик смолистый',
    'трутовик сосновый': 'трутовик окаймленный',
    'трутовик яблоневый': 'трутовик яблоневый',
    'трюфел': 'трюфель',
    'турки': 'колпак кольчатый',
    'фламмулин': 'фламмулина',
    'чаг': 'чага',
    'чайный': 'чайный гриб',
    'чесночник': 'чесночник',
    'чернушк': 'черный груздь',
    'чешутчатк': 'чешутчатка',
    'чинарик': 'вешенка',
    'шампин': 'шампиньон',
    'шиитак': 'шиитаке',
    'шитаки': 'шиитаке',
    'эноки': 'фламмулина',
    'ярсагумба': 'кордицепс'
}

In [16]:
len(mushrooms_dict)

154

In [17]:
%%time

# циклом выбираю нужные строки


work_mushrooms_ula = pd.DataFrame()

for mushroom in all_mushrooms:
    temp_1 = mushrooms_ula[(mushrooms_ula.topic.str.contains(mushroom, case=False))]
    temp_2 = mushrooms_ula[(mushrooms_ula.ads_text.str.contains(mushroom, case=False, na=False))]
    
    work_mushrooms_ula = work_mushrooms_ula.append(temp_1, ignore_index=True)
    work_mushrooms_ula = work_mushrooms_ula.append(temp_2, ignore_index=True)

Wall time: 677 ms


In [18]:
work_mushrooms_ula.shape

(905, 7)

In [19]:
# удаляю дубликаты

work_mushrooms_ula = work_mushrooms_ula.drop_duplicates()

In [20]:
work_mushrooms_ula.shape

(823, 7)

In [21]:
# сохраняю в файл, чтобы дочистить вручную

#work_mushrooms_ula.to_excel('C:/00_Projects/Rus_mushrooms/ula.xlsx', index=False)

In [22]:
# сохраняю в файл, чтобы дочистить вручную

temp = pd.read_excel('C:/00_Projects/Rus_mushrooms/ula.xlsx', sheet_name='work_ula')

In [31]:
temp.topic = temp.topic.str.replace('  ', ' ')
temp.topic = temp.topic.str.lower()

In [32]:
# функция, которая указывает виды грибов в объявлениях

def find_need_root(column):
    words = []
    for root in all_mushrooms:
        if root in str(column.topic):
            words += [mushrooms_dict[root]]
            
    words = list(set(words))
    words.sort()
    mushrooms = ' '.join(words)
    
    return words

In [33]:
%%time

# добавляю стобец с видами грибов в объявлении

temp['mushrooms'] = temp.apply(find_need_root, axis=1)

Wall time: 646 ms


In [36]:
# группирую по id продавца и суммирую виды грибов

temp_1 = (temp
          .groupby(['user_id'], as_index=False)
          .agg({'mushrooms':'sum'})
         )

In [37]:
# функция для удаления повторов

def drop_duplicates_mushrooms(df):
    new = list(set(df.mushrooms))
    new.sort()
    
    return new

In [39]:
temp_1['mushroom_types'] = temp_1.apply(drop_duplicates_mushrooms, axis=1)

In [40]:
#temp_1.to_excel('C:/00_Projects/Rus_mushrooms/temp.xlsx', index=False)